## Linear regression with interaction effects

Estimating a linear regression model with annual income in USD (**earn**) as the response variable. Investigating whether the relationship between income **earn** and level of education **education** differs depending on gender **male** or not.

### Data Understanding

Y (response variable) = **earn** (income in USD)

X1 = **education** (level of education, number of years)

X2 = **male** (gender, male or female)

X3 = Interaction term between **education** and **male** (education × male)

Dataset Source: [https://github.com/avehtari/ROS-Examples/blob/master/Earnings/data/earnings.csv]

In [1]:
import pandas as pd


In [4]:
dataset_orig = pd.read_csv("Datasets/earnings.csv")

In [5]:
dataset_orig.head()

,height,weight,male,earn,earnk,ethnicity,education,mother_education,father_education,walk,exercise,smokenow,tense,angry,age
0,74,210.0,1,50000.0,50.0,White,16.0,16.0,16.0,3,3,2.0,0.0,0.0,45
1,66,125.0,0,60000.0,60.0,White,16.0,16.0,16.0,6,5,1.0,0.0,0.0,58
2,64,126.0,0,30000.0,30.0,White,16.0,16.0,16.0,8,1,2.0,1.0,1.0,29
3,65,200.0,0,25000.0,25.0,White,17.0,17.0,NaN,8,1,2.0,0.0,0.0,57
4,63,110.0,0,50000.0,50.0,Other,16.0,16.0,16.0,5,6,2.0,0.0,0.0,91


### Data Preparation
**earn** - the person's annual income in USD. Retrieved from registry.

**height** - height of the person in inches. Self-reported.

**weight** - weight of the person in pounds. Self-reported.

**male** - dummy variable with values ​​1 (male) and 0 (female). Retrieved from registry.

**ethnicity** - the person's ethnicity, with Black, Hispanic, Other, and White categories. Retrieved from registry.

**education** - number of years of education. Retrieved from registry.

**age** - age. Retrieved from registry

In [6]:
dataset = dataset_orig[['earn', 'height', 'weight', 'male', 'ethnicity', 'education', 'age']]

In [7]:
dataset.head()

,earn,height,weight,male,ethnicity,education,age
0,50000.0,74,210.0,1,White,16.0,45
1,60000.0,66,125.0,0,White,16.0,58
2,30000.0,64,126.0,0,White,16.0,29
3,25000.0,65,200.0,0,White,17.0,57
4,50000.0,63,110.0,0,Other,16.0,91
